## Kompletny przykład
### Przykładowy wycinek aplikacji wykonującej podsumowanie transakcji pobranych z jakiegoś źródła zewnętrznego

### Klient bazodanowy

In [ ]:
from __future__ import annotations
from typing import List
from decimal import Decimal
from dataclasses import dataclass
from enum import Enum

from reader import Reader
from utils import cmap, cfilter, pipe
from base import DomainError
import result


@dataclass
class DbClient:
    dummy_transactions: List[tuple]

    def get_transactions(self) -> result.Result[List[tuple], str]:
        # Tutaj w realnej aplikacji potrencjalnie mógłby wystąpić jakiś błąd - zwrócony zostałby wtedy Error
        return result.Ok(self.dummy_transactions)

### Definicja dziedziny

In [ ]:
@dataclass(frozen=True, repr=False)
class ValidationError(DomainError): pass

@dataclass(frozen=True)
class Transaction:
    @dataclass(frozen=True)
    class Id:
        value: str

        @staticmethod
        @result.from_generator
        def create(value: str) -> result.Result[Transaction.Id, ValidationError]:
            if not value.startswith("#"):
                return ValidationError.create("Id should start with #")
            return value

    class Type(Enum):
        CASH = 'CASH'
        CARD = 'CARD'

        @staticmethod
        @result.from_generator
        def create(_type: str) -> result.Result[Transaction.Type, ValidationError]:
            try:
                return Transaction.Type[_type]
            except KeyError:
                return ValidationError.create(f"Invalid type of transaction given: {_type}")

    id: Transaction.Id
    value: Decimal
    type: Transaction.Type

    @staticmethod
    @result.from_generator
    def create(_id: str, value: Decimal, _type: str) -> result.Result[Transaction, ValidationError]:
        t_id, t_type = yield result.aggregate([
            Transaction.Id.create(_id),
            Transaction.Type.create(_type)
        ]).flat_map_error(ValidationError.join_errors(
            "Transaction validation errors. "
            f"Raw data (id={_id!r}, value={value!r}, type={_type!r})"
        ))
        return Transaction(t_id, value, t_type)


### Utworzenie procesu zawierającego logikę aplikacji
**To jest kod definiowany w obszarze logiki biznesowej.  
Na tym etapie wszystkie funkcje są czyste, nieświadome zależności i gotowe na obsługę błędów powstałych w trakcie wykonania**

In [ ]:
@Reader.create
def fetch_transactions(trans_type: str):
    db_client = yield lambda env: env["db_client"]
    return (
        db_client.get_transactions()
        | cmap(lambda trans_data: Transaction.create(*trans_data))
        | result.aggregate()
        | cfilter(lambda t: t.type == Transaction.Type[trans_type])
        | list
    )

def summarize(transactions: List[Transaction]) -> str:
    return pipe(
        transactions,
        cmap(lambda t: t.value),
        sum,
        (lambda value: f"REPORT RESULT. TRANSACTION SUM IS: {value}")
    )

complete_process = (
    fetch_transactions("CASH")
    | result.flat_map(summarize)
    | result.map_error(lambda e: f"Could not produce report because of error: {e}")
)

### Uruchomienie procesu
**W górnych warstwach aplikacji, gdzie dostępne są zależności wymagane przez proces,  
tworzony jest obiekt środowiska i podawany do procesu, co powoduje jego uruchomienie  
Na koniec badany jest wynik działania procesu przez dopasowanie do obu wariantów**

In [ ]:
env = {
    "db_client": DbClient([
        ("#1234", Decimal("11.23"), "CASH"),
        # ("1234", Decimal("-12.34"), "PAYPAL"),
        ("#1234", Decimal("12.44"), "CASH")
    ])
}
process_result = complete_process.run(env)

process_result.match(
    lambda report: print("Process was successful.", report),
    lambda error: print("Process failed.", error)
)